In [3]:
import pandas as pd
import json
 


In [10]:
lines = []
with open('absa_textcat.jsonl', 'r') as f:
    for line in f:
        lines.append(json.loads(line))

In [20]:
int(lines[0]['accept']==['POSITIVE'])

1

In [23]:
shuffle(lines)

[{'text': 'Received very promptly',
  '_input_hash': -1907126329,
  '_task_hash': 1750193916,
  'options': [{'id': 'POSITIVE', 'text': 'POSITIVE'},
   {'id': 'NEGATIVE', 'text': 'NEGATIVE'}],
  '_session_id': None,
  '_view_id': 'choice',
  'accept': ['POSITIVE'],
  'config': {'choice_style': 'multiple'},
  'answer': 'accept',
  '_timestamp': 1636036835},
 {'text': 'abit on the translucent side',
  '_input_hash': -1120954007,
  '_task_hash': 1162046537,
  'options': [{'id': 'POSITIVE', 'text': 'POSITIVE'},
   {'id': 'NEGATIVE', 'text': 'NEGATIVE'}],
  '_session_id': None,
  '_view_id': 'choice',
  'accept': ['NEGATIVE'],
  'config': {'choice_style': 'multiple'},
  'answer': 'accept',
  '_timestamp': 1636039295},
 {'text': 'Quality seems ok ok',
  '_input_hash': 957454630,
  '_task_hash': -1321818284,
  'options': [{'id': 'POSITIVE', 'text': 'POSITIVE'},
   {'id': 'NEGATIVE', 'text': 'NEGATIVE'}],
  '_session_id': None,
  '_view_id': 'choice',
  'accept': [],
  'config': {'choice_style'

In [40]:
def dataHandler(lines,row):
    return (lines[row]['text'],{'cats': {'neg': lines[row]['accept']==['NEGATIVE'], 'pos': lines[row]['accept']==['POSITIVE']}})
from sklearn.utils import shuffle
def load_training_data(lines,split):
    # Load from files
    lines = shuffle(lines)
    datas=[]
    for i in range(len(lines)):
        datas.append(dataHandler(lines,i))

    split = int(len(datas) * split)
    return datas[:split], datas[split:]

training_data, testing_data =load_training_data(lines,0.8)

In [41]:
len(training_data)

1086

In [42]:
import random
import spacy
from spacy.util import minibatch, compounding

def train_model(
    training_data: list,
    test_data: list,
    iterations: int = 20
) -> None:
    # Build pipeline
    nlp = spacy.load("en") # Change this for loading medium or large models
    if "textcat" not in nlp.pipe_names:
        textcat = nlp.create_pipe(
            "textcat", config={"architecture": "simple_cnn"}
        )
        nlp.add_pipe(textcat, last=True)
    else:
        textcat = nlp.get_pipe("textcat")

    textcat.add_label("pos")
    textcat.add_label("neg")

    # Train only textcat
    training_excluded_pipes = [
        pipe for pipe in nlp.pipe_names if pipe != "textcat"
    ]
    with nlp.disable_pipes(training_excluded_pipes):
        optimizer = nlp.begin_training()
        # Training loop
        print("Beginning training")
        print("Loss\tPrecision\tRecall\tF-score")
        batch_sizes = compounding(
            4.0, 32.0, 1.001
        )  # A generator that yields infinite series of input numbers
        for i in range(iterations):
            print(f"Training iteration {i}")
            loss = {}
            random.shuffle(training_data)
            batches = minibatch(training_data, size=batch_sizes)
            for batch in batches:
                text, labels = zip(*batch)
                nlp.update(text, labels, drop=0.2, sgd=optimizer, losses=loss)
            with textcat.model.use_params(optimizer.averages):
                evaluation_results = evaluate_model(
                    tokenizer=nlp.tokenizer,
                    textcat=textcat,
                    test_data=test_data
                )
                print(
                    f"{loss['textcat']}\t{evaluation_results['precision']}"
                    f"\t{evaluation_results['recall']}"
                    f"\t{evaluation_results['f-score']}"
                )

    # Save model
    with nlp.use_params(optimizer.averages):
        nlp.to_disk("model_artifacts_shopee_aspect")

In [43]:
def evaluate_model(
    tokenizer, textcat, test_data: list
) -> dict:
    reviews, labels = zip(*test_data)
    reviews = (tokenizer(review) for review in reviews)
    true_positives = 0
    false_positives = 1e-8  # Can't be 0 because of presence in denominator
    true_negatives = 0
    false_negatives = 1e-8
    for i, review in enumerate(textcat.pipe(reviews)):
        true_label = labels[i]['cats']
        for predicted_label, score in review.cats.items():
            # Every cats dictionary includes both labels. You can get all
            # the info you need with just the pos label.
            if (
                predicted_label == "neg"
            ):
                continue
            if score >= 0.5 and true_label["pos"]:
                true_positives += 1
            elif score >= 0.5 and true_label["neg"]:
                false_positives += 1
            elif score < 0.5 and true_label["neg"]:
                true_negatives += 1
            elif score < 0.5 and true_label["pos"]:
                false_negatives += 1
    precision = true_positives / (true_positives + false_positives)
    recall = true_positives / (true_positives + false_negatives)

    if precision + recall == 0:
        f_score = 0
    else:
        f_score = 2 * (precision * recall) / (precision + recall)
    return {"precision": precision, "recall": recall, "f-score": f_score}

In [44]:
train_model(training_data, testing_data, 12)

Beginning training
Loss	Precision	Recall	F-score
Training iteration 0
5.220747978892428	0.9364161849169701	0.9152542372364263	0.9257142856613877
Training iteration 1
1.4472333948604887	0.8776595744214011	0.9322033897778416	0.9041095889915557
Training iteration 2
0.5934133825611575	0.8972972972487947	0.9378531072916467	0.9171270717725344
Training iteration 3
0.3261954931292865	0.9111111110604938	0.9265536722640365	0.9187675069513295
Training iteration 4
0.13723778962554434	0.9111111110604938	0.9265536722640365	0.9187675069513295
Training iteration 5
0.15779360050805913	0.9166666666157407	0.9322033897778416	0.9243697478473742
Training iteration 6
0.05895317923916912	0.9116022098943866	0.9322033897778416	0.9217877094457101
Training iteration 7
0.03928588918474851	0.9269662920827546	0.9322033897778416	0.9295774647363617
Training iteration 8
0.026889019096559608	0.9213483145549803	0.9265536722640365	0.9239436619197777
Training iteration 9
0.01871498061160537	0.9209039547502313	0.92090395475

In [53]:
TEST_REVIEW='simply too small'
def test_model(input_data: str = TEST_REVIEW):
    #  Load saved trained model
    loaded_model = spacy.load("model_artifacts_shopee_aspect")
    # Generate prediction
    parsed_text = loaded_model(input_data)
    # Determine prediction to return
    if parsed_text.cats["pos"] > parsed_text.cats["neg"]:
        prediction = "Positive"
        score = parsed_text.cats["pos"]
    else:
        prediction = "Negative"
        score = parsed_text.cats["neg"]
    print(f"Review text: {input_data}\nPredicted sentiment: {prediction}"
        f"\tScore: {score}")
    

    
test_model(TEST_REVIEW)

Review text: simply too small
Predicted sentiment: Negative	Score: 0.9999030828475952
